In [1]:
#Eliminamos los frames de un proceso anterior para limpiar las carpetas
!rm -rf inputs/*
!rm -rf resultados_threshold_pre/*
!rm -rf resultados_CLAHE/*
!rm -rf resultados_threshold_post/*
!rm -rf results/*

In [1]:
#########################################################################################
# Funciones
#########################################################################################
  
import cv2 
import numpy as np
import os
import glob # libreria para buscar los archivos a procesar
import torch 
from tqdm import tqdm
import re

def frames_to_video(input_folder, output_file='output.mp4', fps=30, frame_pattern='frame_*.jpg'):
    """
    Convierte una secuencia de imágenes (frames) a un archivo de video MP4.
    
    Args:
        input_folder (str): Ruta a la carpeta que contiene las imágenes
        output_file (str): Nombre del archivo de salida (default: 'output.mp4')
        fps (int): Frames por segundo del video resultante (default: 30)
        frame_pattern (str): Patrón para buscar los archivos de frames (default: 'frame_*.jpg')
    
    Returns:
        bool: True si la conversión fue exitosa, False en caso contrario
    """
    # Obtener la lista de todos los frames en la carpeta
    frames_path = os.path.join(input_folder, frame_pattern)
    frames = sorted(glob.glob(frames_path), key=lambda x: int(re.findall(r'\d+', os.path.basename(x))[0]))
    
    if not frames:
        print(f"No se encontraron frames con el patrón {frame_pattern} en {input_folder}")
        return False
    
    # Leer el primer frame para obtener las dimensiones
    frame = cv2.imread(frames[0])
    if frame is None:
        print(f"No se pudo leer el frame: {frames[0]}")
        return False
    
    height, width, layers = frame.shape
    
    # Definir el codec y crear el objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec MP4
    video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
    
    # Agregar cada frame al video
    for frame_file in frames:
        frame = cv2.imread(frame_file)
        if frame is not None:
            video.write(frame)
        else:
            print(f"Error al leer el frame: {frame_file}")
    
    # Liberar el objeto VideoWriter
    video.release()
    print(f"Video creado exitosamente: {output_file}")
    
    return True

def recorte_auto(input_video_path, output_folder, output_video_path, umbral):
    """
    Procesa un video de una piscina de metal fundido en movimiento horizontal,
    detecta los límites vertical y horizontal donde aparece la piscina entre el
    primer y último frame, recorta todos los frames a estos límites y 
    genera un nuevo video.
    
    Args:
        input_video_path: Ruta del video de entrada
        output_folder: Carpeta donde guardar los frames recortados
        output_video_path: Ruta donde guardar el video resultante
        umbral: Valor de pixel que sirve como umbral entre 0 y 255 
    """
    # Crear carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Abrir el video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el video {input_video_path}")
        return
    
    # Obtener propiedades del video
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Procesando video: {frame_count} frames, {width}x{height} píxeles, {fps} FPS")
    
    # Variables para almacenar los límites verticales
    min_y = height
    max_y = 0
    
    # Variables para almacenar los límites horizontales
    # Inicializamos con valores que garantizan que cubran toda la anchura inicialmente
    min_x_global = width
    max_x_global = 0
    
    # Primera pasada: Detectar contornos en todos los frames para determinar límites verticales
    # print("Pasada 1: Detectando límites verticales...")
    
    for _ in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convertir a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Aplicar umbral fijo de 60
        _, thresh = cv2.threshold(gray, umbral, 255, cv2.THRESH_BINARY)
        
        # Detectar contornos
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Elegir el contorno más grande (asumimos que es la piscina)
            c = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(c)
            
            # Actualizar los límites verticales globales
            min_y = min(min_y, y)
            max_y = max(max_y, y + h)
    
    # Reiniciar el video para detectar los límites horizontales en el primer y último frame
    # print("Detectando límites horizontales en el primer y último frame...")
    cap.release()
    cap = cv2.VideoCapture(input_video_path)
    
    # Obtener primer frame
    ret, first_frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el primer frame")
        return
    
    # Procesar primer frame para obtener límites horizontales
    gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, umbral, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        min_x_first = x
        max_x_first = x + w
        # print(f"Límites horizontales en el primer frame: x_min={min_x_first}, x_max={max_x_first}")
    else:
        min_x_first = 0
        max_x_first = width
    
    # Posicionar en el último frame
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
    
    # Obtener último frame
    ret, last_frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el último frame")
        return
    
    # Procesar último frame para obtener límites horizontales
    gray = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, umbral, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)
        min_x_last = x
        max_x_last = x + w
        # print(f"Límites horizontales en el último frame: x_min={min_x_last}, x_max={max_x_last}")
    else:
        min_x_last = 0
        max_x_last = width
    
    # Determinar los límites horizontales globales
    # Extendemos la región para que incluya la posición de la piscina tanto en el inicio como en el final
    min_x_global = min(min_x_first, min_x_last)
    max_x_global = max(max_x_first, max_x_last)
    
    # Añadir un margen de seguridad
    margin = 3
    min_y = max(0, min_y - margin)
    max_y = min(height - 1, max_y + margin)
    min_x_global = max(0, min_x_global - margin)
    max_x_global = min(width - 1, max_x_global + margin)
    
    # print(f"Límites finales para recorte: y_min={min_y}, y_max={max_y}, x_min={min_x_global}, x_max={max_x_global}")
    
    # Reiniciar el video para la tercera pasada (recorte y guardado)
    cap.release()
    cap = cv2.VideoCapture(input_video_path)
    
    # Tercera pasada: recortar y guardar los frames
    # print("Pasada 3: Recortando frames y guardando...")
    
    # Dimensiones del frame recortado
    new_height = max_y - min_y + 1
    new_width = max_x_global - min_x_global + 1
    
    # Preparar el escritor de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (new_width, new_height))
    
    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recortar el frame según los límites detectados
        cropped_frame = frame[min_y:max_y+1, min_x_global:max_x_global+1]

        # Redimensionar para mantener una salida de tamaño constante
        #cropped_frame = cv2.resize(cropped_frame, (width, height))
        
        # Guardar el frame recortado
        output_path = os.path.join(output_folder, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(output_path, cropped_frame)
        
        # Añadir al video de salida
        out.write(cropped_frame)
        
        frame_number += 1
    
    # Liberar recursos
    cap.release()
    out.release()
    
    print(f"Recorte completado.\n")
    # print(f"Dimensiones del recorte: {new_width}x{new_height} píxeles")
    # print(f"Video guardado en: {output_video_path}")
    # print(f"Frames guardados en: {output_folder}")


def umbral(input_folder, output_folder, umbral_pre):
    """
    Aplica filtro de umbralización para eliminar el ruido en las regiones fuera de la piscina

    Args:
    input_folder: ruta a carpeta con frames a procesar (string)
    output_folder: ruta a carpeta con frames procesados (string)
    umbral: valor de umbral entre 0 (negro) y 255 (blanco) (entero)
    """
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Obtener la lista de archivos en la carpeta de entrada
    # Se asume que las imágenes pueden tener extensión jpg, png, etc.
    image_paths = glob.glob(os.path.join(input_folder, "*.*"))
    
    for img_path in image_paths:
        # Leer la imagen en escala de grises
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"No se pudo cargar la imagen: {img_path}")
            continue
    
        # Aplicar el umbral TOZERO
        # Los pixeles con valor menor a umbral2 se pondrán a 0 y los mayores se mantendrán
        _, thresh_img = cv2.threshold(img, umbral_pre, 255, cv2.THRESH_TOZERO)
    
        # Generar la ruta de salida usando el mismo nombre de archivo
        filename = os.path.basename(img_path)
        output_path = os.path.join(output_folder, filename)
    
        # Guardar la imagen procesada
        cv2.imwrite(output_path, thresh_img)
        #print(f"Procesada y guardada: {output_path}")
    
    print("Threshold completado.")

def CLAHE(input_folder, output_folder, tg, cl):
    """
    Aplica ecualizaciones por bloques (kernel) y une los resultados 
    de cada bloque en una sola imagen con interpolacion en los bordes.

    Args:
    input_folder: ruta a carpeta con frames a procesar (string)
    output_folder: ruta a carpeta con frames procesados (string)
    tg: Tile Grid. Tamaño del kernel cuadrado de tg x tg (entero)
    cl: Clip LImit. 
    """
    # input_folder = "resultados_threshold"
    # output_folder = f"resultados_CLAHE_{tg_cont}_{cl_cont}"
        
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Crear objeto CLAHE con parámetros (clipLimit y tileGridSize pueden ajustarse)
    clahe = cv2.createCLAHE(clipLimit=cl, tileGridSize=(tg, tg))
        
    # Obtener la lista de archivos de imagen en la carpeta de entrada
    image_paths = glob.glob(os.path.join(input_folder, "*.*"))
        
    # Procesar cada imagen
    for img_path in image_paths:
        # Cargar la imagen
        img = cv2.imread(img_path)
        if img is None:
            print(f"No se pudo cargar la imagen: {img_path}")
            continue
        
        # Convertir la imagen a escala de grises (CLAHE trabaja con imágenes en grises)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Aplicar CLAHE
        processed = clahe.apply(gray)
        
        # Generar la ruta de salida manteniendo el mismo nombre de archivo
        filename = os.path.basename(img_path)
        output_path = os.path.join(output_folder, filename)
        
        # Guardar la imagen procesada
        cv2.imwrite(output_path, processed)
        #print(f"Procesada y guardada: {output_path}")
        
    print("CLAHE completado.\n")
    

In [7]:
#########################################################################################
# Recorte 
#########################################################################################
  

# Obtener la ruta del directorio de trabajo actual (normalmente es la carpeta donde se ejecuta el notebook)
#directorio_actual = os.getcwd()
input_video = "/home/riemann007/Documentos/Proyecto CIDESI/polvo_completo_original.avi" 
#output_frames_folder = os.path.join(directorio_actual,"inputs")
output_frames_folder = "inputs"
umbral_recorte = 70
#output_video = os.path.join(directorio_actual,"video_recortado_xy_pl3.mp4")
output_video = f"/videos/FULL_umbral{umbral_recorte}_pl4.mp4"


# Procesar el video
recorte_auto(input_video, output_frames_folder, output_video, umbral_recorte)

frames_to_video("inputs","/videos/recorte_FULL.mp4")

Procesando video: 14108 frames, 1024x512 píxeles, 30.303030303030305 FPS


100%|██████████| 14108/14108 [00:42<00:00, 334.17it/s]


Recorte completado.



In [3]:
#########################################################################################
# THRESHOLD preCLAHE
#########################################################################################

#PARAMETROS:
umbral_pre = 55

# Carpetas de entrada y salida
input_folder = "inputs"
output_folder = "resultados_threshold_pre"

# Umbralización pre CLAHE
umbral(input_folder, output_folder, umbral_pre)

# Video para observar resultados
ruta_frames = "resultados_threshold_pre"
ruta_video = f"videos/FULL_pre{umbral_pre}.mp4"
frames_to_video(ruta_frames, ruta_video)


Threshold completado.
Video creado exitosamente: videos/FULL_pre55.mp4


True

In [4]:
#########################################################################################
# CLAHE (Ecualizacion adaptativa) 
#########################################################################################


#PARAMETROS
tg = 8 #Tile Grid: tamaño del kernel (kernel cuadrado)
cl = 5 #CLip limit:

# Carpetas de entrada y salida
input_folder = "resultados_threshold_pre"
output_folder = f"resultados_CLAHE_{tg}_{cl}"


# Ecualizacion adaptativa (CLAHE)
CLAHE(input_folder,output_folder, tg, cl)


# Creacion de video CLAHE
ruta_frames = f"resultados_CLAHE_{tg}_{cl}"
ruta_video = f"videos/FULL_CLAHE_{tg}_{cl}.mp4"
fps = 30
        
frames_to_video(ruta_frames, ruta_video, fps)

CLAHE completado.

Video creado exitosamente: videos/FULL_CLAHE_8_5.mp4


True

In [5]:
#########################################################################################
# THRESHOLD postCLAHE
#########################################################################################

#PARAMETROS:
umbral_post = 30

# Carpetas de entrada y salida
input_folder = f"resultados_CLAHE_{tg}_{cl}"
output_folder = "resultados_threshold_post"

# Umbralización pre CLAHE
umbral(input_folder, output_folder, umbral_post)

# Video para observar resultados
ruta_frames = "resultados_threshold_post"
ruta_video = f"videos/FULL_post{umbral_post}.mp4"
frames_to_video(ruta_frames, ruta_video)

Threshold completado.
Video creado exitosamente: videos/FULL_post30.mp4


True

In [6]:
# Verificamos disponibilidad de GPU

print(torch.cuda.is_available())  # Imprime True si CUDA esta disponible
print(torch.cuda.device_count())  # Muestra el número de GPUs disponibles
print(torch.cuda.get_device_name(0))  # Muestra el modelo de GPU en caso de existir



#########################################################################################
# Super resolucion con modelo REal-ESRGAN
#########################################################################################

#Hace la inferencia (ejecuta el modelo para los frames del video en la carpeta llamada "inputs")

#Si es en cpu usar esta linea y comentar la de abajo
#!python inference_realesrgan.py -n RealESRGAN_x4plus -i resultados_threshold_post -dn 0

#Si es en gpu nvidia usar esta linea y comentar la de arriba
# !python inference_realesrgan.py -n RealESRGAN_x4plus -i resultados_threshold_post -dn 0 --fp32 --tile 200 --gpu-id 0


#########################################################################################
# Super resolucion con modelo DRCT 
#########################################################################################

# Si detecta GPU carga el modelo directo en ella
!python inference.py --input resultados_threshold_post --output results --model_path "/home/riemann007/JupyterLab/Proyecto CIDESI/DRCT/drct/models/net_g_latest.pth"


#########################################################################################
# Super resolucion con modelo RVRT
#########################################################################################

# !rm -r RVRT
# # Clone RVRT
# !git clone https://github.com/JingyunLiang/RVRT.git
# %cd RVRT
# !pip install -r requirements.txt 


# !python main_test_rvrt.py --task 001_RVRT_videosr_bi_REDS_30frames --folder_lq testsets/uploaded --tile 0 128 128 --tile_overlap 2 20 20 --num_workers 2 --save_result

########################################################################################


# Video con super resolucion
# frames_folder = os.path.join(directorio_actual, "results")  # Ruta a carpeta de frames
frames_folder = "results"
# output_path =  os.path.join(directorio_actual,"videos/video_sr_auto_pl3.mp4") # Ruta a carpeta de guardado y nombre del video
output_path = f"videos/FULL_sr_{tg}_{cl}.mp4"
frames_to_video(frames_folder, output_path, fps, frame_pattern='frame_*.png')

print("\nAumento de resolución x4 realizada.\n")



print("\nProceso terminado.\n")

True
1
NVIDIA GeForce GTX 1050
/home/riemann007/miniconda3/envs/drct_env/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
DRCT(
  (conv_first): Conv2d(3, 180, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (patch_embed): PatchEmbed(
    (norm): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
  )
  (patch_unembed): PatchUnEmbed()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): RDG(
      (swin1): SwinTransformerBlock(
        dim=180, input_resolution=(64, 64), num_heads=6, window_size=16, shift_size=0, mlp_ratio=2
        (norm1): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
        (attn): WindowAttention(
          dim=180, window_size=(16, 16), num_heads=6
          (qkv): Linear(in_fea